### Библиотеки и настройки

In [ ]:
# для запросов
import requests

# для работы с JSON
import json

# для работы с папками
import os

# 
from datetime import timedelta, date

# для дафреймов
import pandas as pd

In [ ]:
# пути к папкам

file_path = 'C:/00_Data/test/'

In [ ]:
# рекурсия для поиска значений

def recurs_find_key(key, obj):
    if key in obj:
        return obj[key]
    
    for k, v in obj.items():
        if type(v) == dict:
            result = recurs_find_key(key, v)
            
            if result is not None:
                return result 

In [ ]:
# функция запроса

def get_contracts(**kwargs):
    
    try:
        url = 'http://openapi.clearspending.ru/restapi/v3/contracts/search/'
        params = {**kwargs}
        data = requests.get(url, params).json()
        
    except:
        print('Не отдает данные')
    
    return data

In [ ]:
def get_week(day=None):
    
    if day == None:
        start_date = date.today()
        
    else:
        start_date = datetime.datetime.strptime(day, "%d.%m.%Y" )
        
    end_date = start_date - timedelta(weeks=1)
    
    date_range = f'{end_date.strftime("%d.%m.%Y")}-{start_date.strftime("%d.%m.%Y")}'
    
    return date_range

In [ ]:
region_list = []

for i in range(80):
    if i < 10:
        region_list += ['0' + str(i)]
        
    else:
        region_list += [str(i)]

In [ ]:
region_list += ['83', '86', '87', '89', '91', '92']

In [ ]:
print(region_list)

In [ ]:
len(region_list)

In [ ]:
url = 'http://openapi.clearspending.ru/restapi/v3/contracts/search/?customerregion=78&productsearch=Маска медицинская&daterange=01.01.2021-31.12.2021'

In [ ]:
raw_data = requests.get(url).json()

In [ ]:
with open(file_path + 'raw_data.json', 'w') as json_file:
    json.dump(raw_data, json_file)    

In [ ]:
for contract in all_data[:10]:
    print(recurs_find_key('products', contract))

In [ ]:
all_data = raw_data['contracts']['data']

In [ ]:
data = [] # список для данных

for contract in all_data:
    
    # номер ФЗ
    fz_number = recurs_find_key('fz', contract)
    
    # ссылка на контракт
    contract_url = recurs_find_key('contractUrl', contract)
    
    # номер контракта в реестре Закупок (номер есть в ссылке)
    num_reg = recurs_find_key('regNum', contract)
    
    # дата подписания
    sign_date = recurs_find_key('signDate', contract)
    
    # код региона
    region_code = recurs_find_key('regionCode', contract)
    
    # информация о закачике
    customer = recurs_find_key('customer', contract)
    customer_inn = recurs_find_key('inn', customer)
    customer_name = recurs_find_key('fullName', customer)
    
    # общая цена контракта    
    total_price = recurs_find_key('price', contract)
    
    # информация о поставщиках
    supplier_dict = {}
    try:
        suppliers = recurs_find_key('suppliers', contract)
        
        for supplier in suppliers:
            supplier_inn = recurs_find_key('inn', supplier)
            supplier_name = recurs_find_key('organizationName', supplier)
                        
    except:
        supplier_inn = 'none'
        supplier_name = 'none'
            
    supplier_dict[supplier_inn] = supplier_name
        
        
    products = recurs_find_key('products', contract)
    
    
    data.append([contract_url, sign_date, num_reg,
                 price, region_code,
                 customer, customer_inn, customer_name, supplier_dict, suppliers,
                products])


In [ ]:
headers = ['contract_url', 'sign_date', 'num_reg',
           'price', 'region_code',
           'customer', 'customer_inn', 'customer_name', 'supplier_dict', 'suppliers',
           'products']

work = pd.DataFrame(data, columns=headers)

In [ ]:
work.head(5)